In [1]:
from datetime import datetime, timedelta
from IPython.display import clear_output
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import numpy as np
import pyautogui
import time

pd.options.display.float_format = '{:,.2f}'.format

# Downloads tickers and formatting data

### Fetching tickers from nasdaq and filtering out ETFs are stocks that we have a problem getting their prices

In [2]:
def get_all_tickers(etf=False):
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    all_tickers = all_tickers[~all_tickers["Test Issue"]]  # Filter out stocks that we can't fetch their data for some reason
    
    if not etf:
        all_tickers = all_tickers[~all_tickers["ETF"]]  # Filter out ETFs
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]

### Formatting DataFrame

In [3]:
def get_info(df):
    high = df['High'].max()
    close = df['Close'].iloc[-1]
    pct_to_52 = ((high - close) / high) * 100
    volume = df["Volume"].iloc[-1]
    date_of_high = df['High'].idxmax()
    
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "Percent from 52": pct_to_52, "Volume": volume, "doh": date_of_high, "Volume$": volume * close})

    return new_df

### Downloads data

In [4]:
def get_data(tickers_list):
    return yf.download(tickers_list, period="1y")

In [27]:
s = time.time()
data = get_data(get_all_tickers())
data_cop = data.copy()
print(time.time() - s)

[*********************100%***********************]  7428 of 7428 completed

55 Failed downloads:
- GDSTR: Period '1y' is invalid, must be one of ['1d', '5d']
- MSSAR: Period '1y' is invalid, must be one of ['1d', '5d']
- EFHTR: Period '1y' is invalid, must be one of ['1d', '5d']
- HHGCR: Period '1y' is invalid, must be one of ['1d', '5d']
- FEXDR: Period '1y' is invalid, must be one of ['1d', '5d']
- GBBKR: Period '1y' is invalid, must be one of ['1d', '5d']
- SFB: Period '1y' is invalid, must be one of ['1d', '5d']
- RCA: No data found for this date range, symbol may be delisted
- CSLMR: Period '1y' is invalid, must be one of ['1d', '5d']
- HSPOR: Period '1y' is invalid, must be one of ['1d', '5d']
- MCAFR: Period '1y' is invalid, must be one of ['1d', '5d']
- BRACR: Period '1y' is invalid, must be one of ['1d', '5d']
- TMTCR: Period '1y' is invalid, must be one of ['1d', '5d']
- SAGAR: Period '1y' is invalid, must be one of ['1d', '5d']
- LBBBR: Period '1y' is invalid, must be one of

In [34]:
df = get_info(data)
cop = df.copy()
df

,Close,52wk High,Percent from 52,Volume,doh,Volume$
A,126.87,160.26,20.83,"437,879.00",2022-12-13,"55,553,709.93"
AA,33.80,58.24,41.96,"1,338,402.00",2022-08-26,"45,237,986.58"
AAC,10.65,10.65,0.05,"3,690.00",2023-07-31,"39,280.05"
AACG,1.32,2.64,50.00,"2,305.00",2022-08-18,"3,042.60"
AACI,NaN,12.20,NaN,NaN,2023-01-30,NaN
...,...,...,...,...,...,...
ZVSA,0.17,25.00,99.33,"173,215.00",2022-12-12,"28,840.30"
ZWS,30.21,31.77,4.91,"129,665.00",2022-08-18,"3,917,179.53"
ZYME,7.16,10.80,33.66,"110,629.00",2023-01-19,"792,656.78"
ZYNE,0.35,1.39,74.97,"66,604.00",2022-08-08,"23,171.53"


### Filtering out stocks that doesn't match 

In [35]:
df = cop.copy()
df = df.dropna(subset=['Close'])
vol_thresh = 10000
vold_thresh = 500000
df = df[df['Volume'] >= vol_thresh]
df = df[df['Volume$'] >= vold_thresh]
time_thresh1 = datetime.now() - timedelta(days=90)  # Hit the high between 120 and 200 days ago (3 - 7 months)
time_thresh2 = datetime.now() - timedelta(days=200)  
df = df[(df["doh"] < time_thresh1) & (df["doh"] > time_thresh2)]
df.sort_values(by='Percent from 52', ascending=True).head(40)

,Close,52wk High,Percent from 52,Volume,doh,Volume$
PG,156.97,158.11,0.72,"1,073,261.00",2023-04-21,"168,469,780.48"
MTSI,76.00,76.56,0.72,"182,763.00",2023-03-09,"13,890,901.31"
OSK,105.62,106.66,0.97,"98,951.00",2023-02-03,"10,451,699.38"
UBS,22.05,22.30,1.13,"527,463.00",2023-03-06,"11,627,921.88"
CGBD,15.61,15.89,1.76,"66,341.00",2023-02-28,"1,035,582.99"
SIG,81.93,83.42,1.79,"86,969.00",2023-02-03,"7,125,370.20"
FAF,63.26,64.66,2.17,"62,199.00",2023-02-02,"3,934,708.64"
GNW,6.24,6.40,2.42,"615,633.00",2023-02-15,"3,844,628.01"
XPRO,23.42,24.04,2.56,"97,478.00",2023-03-02,"2,283,422.08"
CMC,56.59,58.09,2.58,"69,189.00",2023-02-03,"3,915,405.52"


# Testing out (manually backtesting)

In [32]:
def get_info_test(df):        
    days_back = 12
    
    _highs = df['High'][:-days_back].max()
    _closes = df['Close'].iloc[-days_back]
    _pct_to_52 = ((_highs - _closes) / _highs) * 100
    # _volume = np.nanmean(df["Volume"].iloc[-days_back])
    _date_of_high = df['High'][:-days_back].idxmax()
    
    highs = df['High'].max()
    date_of_high = df['High'].idxmax()
    closes = df['Close'].iloc[-1]
    volume = df["Volume"].iloc[-3:].mean()  # Calculates the average volume in the last 3 days
    pct_to_52 = ((highs - closes) / highs) * 100
    
    new_df = pd.DataFrame({'Old Close': _closes, 'Old 52wk High': _highs, "Old Percent from 52": _pct_to_52, "Volume": volume, "doh": _date_of_high, "Close": closes, "Prct252": pct_to_52, "high52": highs, "date of break": date_of_high})

    return new_df

In [33]:
df_test = get_info_test(data)
cop_test = df_test.copy()
cop_test

,Old Close,Old 52wk High,Old Percent from 52,Volume,doh,Close,Prct252,high52,date of break
A,127.62,160.26,20.37,"1,333,326.33",2022-12-13,126.87,20.83,160.26,2022-12-13
AA,33.38,58.24,42.68,"3,132,267.33",2022-08-26,33.80,41.96,58.24,2022-08-26
AAC,10.56,10.60,0.38,"21,163.33",2023-07-18,10.65,0.05,10.65,2023-07-31
AACG,1.39,2.64,47.35,"2,835.00",2022-08-18,1.32,50.00,2.64,2022-08-18
AACI,10.51,12.20,13.85,"8,276.50",2023-01-30,NaN,NaN,12.20,2023-01-30
...,...,...,...,...,...,...,...,...,...
ZVSA,0.20,25.00,99.21,"1,000,905.00",2022-12-12,0.17,99.33,25.00,2022-12-12
ZWS,27.33,31.77,13.98,"590,255.00",2022-08-18,30.21,4.91,31.77,2022-08-18
ZYME,7.87,10.80,27.13,"923,709.67",2023-01-19,7.16,33.66,10.80,2023-01-19
ZYNE,0.36,1.39,74.10,"96,968.00",2022-08-08,0.35,74.97,1.39,2022-08-08


### Filtering and sorting

In [7]:
df_test = cop_test.copy()
time_thresh = datetime.now() - timedelta(days=120)
break_time_thresh = datetime.now() - timedelta(days=2)  # How many days ago it broke
vol_thresh = 10000
# print(break_time_thresh)
df_test = df_test[df_test["Old 52wk High"] != df_test["high52"]]
df_test = df_test[df_test["doh"] < time_thresh]
df_test = df_test[df_test["date of break"] > break_time_thresh]
df_test["diff"] = datetime.now() - df_test["date of break"]
pd.options.display.float_format = '{:.2f}'.format
df_test.sort_values(by='Volume', ascending=False).head(20)

NameError: name 'cop_test' is not defined

# Random shit

In [9]:
all_tickers = web.get_nasdaq_symbols()
all_tickers = all_tickers[~all_tickers["ETF"] & ~all_tickers["Test Issue"]]
# all_tickers.loc["O"]
all_tickers

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.00,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.00,False,NaN,AA,AA,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.00,False,NaN,AAC,AAC,False
AAC.U,True,"Ares Acquisition Corporation Units, each consi...",N,,False,100.00,False,NaN,AAC.U,AAC=,False
AAC.W,True,Ares Acquisition Corporation Redeemable Warran...,N,,False,100.00,False,NaN,AAC.WS,AAC+,False
...,...,...,...,...,...,...,...,...,...,...,...
ZVSA,True,"ZyVersa Therapeutics, Inc. - Common Stock",Q,G,False,100.00,False,D,NaN,ZVSA,False
ZWS,True,Zurn Elkay Water Solutions Corporation Common ...,N,,False,100.00,False,NaN,ZWS,ZWS,False
ZYME,True,Zymeworks Inc. - Common Stock,Q,Q,False,100.00,False,N,NaN,ZYME,False


# Import to TradingView using pyautgui

In [10]:
def get_fit_tickers(df, num):
    return df.index[:num].tolist()


def add_to_tv(df):
    # Click plus (1773, 164)
    tickers = get_fit_tickers(df, 100)
    print(tickers)
    pyautogui.click(1773, 164)
    for ticker in tickers:
        pyautogui.write(ticker)
        pyautogui.press("enter")

# Finding stocks that broke 52wk high yesterday

### First DataFrame that contains necessary information

In [22]:
def parse_df(df, old_high=2):
    # old_high- How many days ago it broke high (x or less)

    high = df['High'].max()  # 52 wk high
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((high - close) / high) * 100  # Percent change to 52wk high
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['High'][:-old_high].idxmax()  # 52wk high before 2 days
    doh = df['High'].idxmax()  # 52wk high today
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "% to 52wk": pct_to_52, "Volume": volume, "Date of high": doh, "Old date of high": old_doh, "Volume$": volume * close})

    return new_df

### Filtering and sorting DataFrame

In [14]:
def filter_sort_df(df):
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a high volume but due to low price they have a high volume
    time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the high between 120 and 200 days ago (3 - 7 months)
    time_thresh2 = datetime.now() - timedelta(days=200)  

    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of high"] < time_thresh1) & (df["Old date of high"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of high"] != df["Date of high"]]
    
    return df.sort_values(by='% to 52wk', ascending=True)

In [29]:
full_df = parse_df(data, 1)
cop = full_df.copy()

df = filter_sort_df(cop)
df

,Close,52wk High,% to 52wk,Volume,Date of high,Old date of high,Volume$
SOVO,22.55,22.63,0.38,"11,605,552.00",2023-08-07,2023-05-11,"261,647,170.73"
VRTV,169.07,169.83,0.44,"701,263.00",2023-08-07,2023-02-28,"118,566,039.58"
URI,484.34,487.00,0.55,"423,360.00",2023-08-07,2023-03-06,"205,050,180.85"
SATS,22.23,22.35,0.55,"181,029.00",2023-08-07,2023-02-27,"4,023,876.47"
HAYW,14.86,15.01,0.97,"1,784,865.00",2023-08-07,2023-02-02,"26,532,017.82"
GHM,16.49,17.15,3.85,"104,922.00",2023-08-07,2023-03-07,"1,730,163.76"
HHLA,10.84,11.30,4.07,"96,462.00",2023-08-07,2023-03-03,"1,045,648.09"
OXAC,10.75,17.50,38.57,"310,007.00",2023-08-07,2023-04-17,"3,332,575.25"
VTGN,13.42,24.71,45.70,"21,614,764.00",2023-08-07,2023-02-06,"289,994,483.18"


In [30]:
add_to_tv(df)

['SOVO', 'VRTV', 'URI', 'SATS', 'HAYW', 'GHM', 'HHLA', 'OXAC', 'VTGN']


In [15]:
cop.loc["JBHT"]

Close                            200.11
52wk High                        203.89
% to 52wk                          1.85
Volume                       820,600.00
Date of high        2023-07-27 00:00:00
Old date of high    2023-02-02 00:00:00
Volume$                  164,210,266.50
Name: JBHT, dtype: object